In [14]:
import numpy as np
import xarray as xr
import salishsea_tools.river_202108 as rivers
import pandas as pd

In [15]:
def make_ts_arr(jdepth, data, volume_dims, rho_inv):  # to do xarray multiplication, make sure rename the dims and pass into function
    data_np = data[:, 0:jdepth].values
    data_np[:, :, 300:500, 1:30] = np.nanmean(data_np[:, :, 300:500, 30:59])
    river_arr = rho_inv * np.multiply(volume_dims[:, 0:jdepth], data_np) ## try again as the xarrays? and multiply by tmask so no need nansum (can acutally do that to volume)
    river_sum = np.nansum(np.nansum(np.nansum(river_arr, axis=3), axis=2), axis=1)
    return river_sum

In [16]:
def custom_arange(start, stop, step):
    num_steps = int(np.ceil((stop - start) / step))
    end_value = np.array([stop])
    ranges = np.arange(start, num_steps*step, step)
    return np.concatenate((ranges, end_value))

In [57]:
def river_ts_array(river, start_date, end_date):
    variable = river + '_river'

    ds = xr.open_dataset('/ocean/cdonaldson/research_runs/hindcast_auto/extractions/SalishSeaCast_river_trace_day_avg_one_{}_all_{}_{}.nc'.format(river, start_date, end_date))

    dep_idx = 40
    step_len = 50
    start = 0 
    data = ds[variable]
    time_idxs = custom_arange(start, len(data), step_len)
    river_arr = np.zeros(len(data))

    for i in np.arange(0, len(time_idxs)-1):
        print('{},{}'.format(time_idxs[i], time_idxs[i+1]))
        river_arr[time_idxs[i]:time_idxs[i+1]] = make_ts_arr(dep_idx, ds[variable][time_idxs[i]:time_idxs[i+1]], volume_dims, rho_inv)

    filename = '1d_hindcast_volume_{}_s2012_{}_dep_0_{}.csv'.format(variable, len(data), dep_idx)  # change this for next iterations
    pd.DataFrame(river_arr).to_csv('dye_to_vol/' + filename)
    print(filename)

In [47]:
mesh = xr.open_dataset('/home/sallen/MEOPAR/grid/mesh_mask202108.nc')
# the mask is 1 where there is water, we want the opposite.  The meshmask has an extra dimension, hence the [0]
tmask = 1 - mesh.tmask[0]

In [48]:
volume = (1-tmask) * mesh.e3t_0[0] * mesh.e1t[0] * mesh.e2t[0]
volume_dims = np.array([volume.to_numpy(), volume.to_numpy()])[0:1] ## do this once, pass volume dims into it

rho_inv = (1/1026)

In [52]:
river = 'nanaimo'
variable = river + '_river'
# ds = xr.open_dataset('/ocean/cdonaldson/research_runs/hindcast_auto/extractions/SalishSeaCast_river_trace_day_avg_Fraser_all_20120101_20140101.nc')
ds = xr.open_dataset('/ocean/cdonaldson/research_runs/hindcast_auto/extractions/SalishSeaCast_river_trace_day_avg_one_{}_all_20120101_20160101.nc'.format(river))  # instead of one ds here, can do ds list and "read" it in at the beginning of each loop because it doesn't store in mem
# actually maybe do the concat after... thereby only doing all of the calculation once? it will be produced incrementally

In [53]:
dep_idx = 40
data = ds[variable]
step_len = 50
start = 0
time_idxs = custom_arange(start, len(data), step_len)
river_arr = np.zeros(len(data))

for i in np.arange(0, len(time_idxs)-1):
    print('{},{}'.format(time_idxs[i], time_idxs[i+1]))
    river_arr[time_idxs[i]:time_idxs[i+1]] = make_ts_arr(dep_idx, ds[variable][time_idxs[i]:time_idxs[i+1]], volume_dims, rho_inv)

0,50
50,100
100,150
150,200
200,250
250,300
300,350
350,400
400,450
450,500
500,550
550,600
600,650
650,700
700,750
750,800
800,850
850,900
900,950
950,1000
1000,1050
1050,1100
1100,1150
1150,1200
1200,1250
1250,1300
1300,1350
1350,1400
1400,1450
1450,1462


In [54]:
filename = '1d_hindcast_volume_{}_s2012_{}_dep_0_{}.csv'.format(variable, len(data), dep_idx)
pd.DataFrame(river_arr).to_csv('dye_to_vol/' + filename)
print(filename)

1d_hindcast_volume_nanaimo_river_s2012_1462_dep_0_40.csv


In [58]:
river_ts_array('puntledge', '20120101', '20160101')

0,50
50,100
100,150
150,200
200,250
250,300
300,350
350,400
400,450
450,500
500,550
550,600
600,650
650,700
700,750
750,800
800,850
850,900
900,950
950,1000
1000,1050
1050,1100
1100,1150
1150,1200
1200,1250
1250,1300
1300,1350
1350,1400
1400,1450
1450,1462
1d_hindcast_volume_puntledge_river_s2012_1462_dep_0_40.csv


In [59]:
river_ts_array('salmon', '20120101', '20160101')

0,50


In [ ]:
river_ts_array('homathko', '20120101', '20160101')

In [ ]:
river_ts_array('squamish', '20120101', '20160101')

In [ ]:
river_ts_array('fraser', '20120101', '20160101')

In [ ]:
## add something down here that brings together all the previous files if need be